In [1]:
import networkx as nx
import requests as rq
from routingapp.common.network_stat_utils import get_inter_group_edges, link_with_port_mn_to_hmap

In [2]:
json_graph = rq.get('http://0.0.0.0:8000/graph').json()
graph = nx.json_graph.node_link_graph(json_graph)
inter_group_edges = get_inter_group_edges(graph)
inter_group_edges

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]

In [3]:
link_with_port = link_with_port_mn_to_hmap()
s1, s2 = inter_group_edges[0]
p1 = link_with_port[(f's{s1}', f's{s2}')]['port1']
p2 = link_with_port[(f's{s1}', f's{s2}')]['port2']

In [4]:
deltal_port_stat = rq.get('http://0.0.0.0:8080/delta_port_stat').json()
deltal_port_stat
# link_usage_s1 = ...
# link_usage_s2 = ...

[{'dpid': 6,
  'port_no': 1,
  'tx_packets': 4,
  'rx_packets': 0,
  'tx_bytes': 250,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 2,
  'tx_packets': 23,
  'rx_packets': 19,
  'tx_bytes': 2263,
  'rx_bytes': 1862,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 3,
  'tx_packets': 12,
  'rx_packets': 14,
  'tx_bytes': 1062,
  'rx_bytes': 1381,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 12,
  'tx_bytes': 1002,
  'rx_bytes': 1062,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 0},
 {'dpid': 5,
  'port_no': 2,
  'tx_packets': 32,
  'rx_packets': 30,
  'tx_bytes': 3145,
  'rx_bytes': 2940,
  'rx_error': 0,
  'durr

In [5]:
dps_hmap = {}
for d in deltal_port_stat:
    key = (d['dpid'], d['port_no'])
    dps_hmap[key] = d
    
dps_hmap

{(6, 1): {'dpid': 6,
  'port_no': 1,
  'tx_packets': 4,
  'rx_packets': 0,
  'tx_bytes': 250,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 2): {'dpid': 6,
  'port_no': 2,
  'tx_packets': 23,
  'rx_packets': 19,
  'tx_bytes': 2263,
  'rx_bytes': 1862,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 3): {'dpid': 6,
  'port_no': 3,
  'tx_packets': 12,
  'rx_packets': 14,
  'tx_bytes': 1062,
  'rx_bytes': 1381,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 4): {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 12,
  'tx_bytes': 1002,
  'rx_bytes': 1062,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 0},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 32,
  'rx_packets': 30,
  'tx_bytes': 31

In [6]:
def get_controller_list():
  """Fetches controller list from the API and converts string/int keys to int keys.

  Returns:
    A list of dictionaries with integer keys.
  """
  ctrler_list = rq.get('http://0.0.0.0:8000/controller_list').json()

  # Use list comprehension for concise conversion
  new_ctrler_list = [{int(key): value} for ctrler in ctrler_list for key, value in ctrler.items() if key.isdigit()]

  return new_ctrler_list

get_controller_list()


[{0: {'ip': '0.0.0.0', 'port': 6633}}, {1: {'ip': '0.0.0.0', 'port': 6634}}]

In [7]:
test = rq.get('http://0.0.0.0:8000/controller_list').json()
type(test)

list

In [8]:
from routingapp.common.network_stat_utils import get_sw_ctrler_mapping
RYU_PORT = 8080
def get_all_delta_port_stat():
    ''' Get delta port stat of all controllers in the network
    Return:
    hashmap of (dpid, port):{delta_port_stat}
    '''
    deltal_port_stat = []
    ctrler_list = get_controller_list()
    
    for ctrler in ctrler_list:
        for key, value in ctrler.items():
            deltal_port_stat += rq.get(f'http://{value.get("ip")}:{RYU_PORT+key}/delta_port_stat').json()

    for d in deltal_port_stat:
        key = (d['dpid'], d['port_no'])
        dps_hmap[key] = d

    return dps_hmap

get_all_delta_port_stat()

{(6, 1): {'dpid': 6,
  'port_no': 1,
  'tx_packets': 4,
  'rx_packets': 0,
  'tx_bytes': 250,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 2): {'dpid': 6,
  'port_no': 2,
  'tx_packets': 23,
  'rx_packets': 19,
  'tx_bytes': 2263,
  'rx_bytes': 1862,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 3): {'dpid': 6,
  'port_no': 3,
  'tx_packets': 12,
  'rx_packets': 14,
  'tx_bytes': 1062,
  'rx_bytes': 1381,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (6, 4): {'dpid': 6,
  'port_no': 4,
  'tx_packets': 11,
  'rx_packets': 12,
  'tx_bytes': 1002,
  'rx_bytes': 1062,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 1000000},
 (5, 1): {'dpid': 5,
  'port_no': 1,
  'tx_packets': 3,
  'rx_packets': 0,
  'tx_bytes': 180,
  'rx_bytes': 0,
  'rx_error': 0,
  'durration_sec': 3,
  'duration_nsec': 0},
 (5, 2): {'dpid': 5,
  'port_no': 2,
  'tx_packets': 32,
  'rx_packets': 30,
  'tx_bytes': 31

In [9]:
get_inter_group_edges(graph)

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]

In [10]:
inter_group_edges = get_inter_group_edges(graph)
deltal_port_stat = get_all_delta_port_stat()
link_with_port = link_with_port_mn_to_hmap()

ige = inter_group_edges[0]

link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port1']
link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port2']

3

In [19]:
import multiprocessing
RYU_PORT = 8080
RESTHOOKMN_PORT = 8000
def get_link_traffic_multi_controller():
    num_ctrler = rq.get(f'http://0.0.0.0:{RESTHOOKMN_PORT}/controller_list').json()
    lqcs = []
    for i in range(len(num_ctrler)):
            lqcs += rq.get(f'http://0.0.0.0:{RYU_PORT}/link_quality').json()
    return lqcs
get_link_traffic_multi_controller()

[{'src.dpid': 6,
  'dst.dpid': 2,
  'delay': 26.80039405822754,
  'packet_loss': 0.0,
  'link_usage': 0.00600866377874024,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 5,
  'delay': 62.60025501251221,
  'packet_loss': 0.0,
  'link_usage': 0.006184605131622841,
  'free_bandwidth': None},
 {'src.dpid': 3,
  'dst.dpid': 1,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 3,
  'dst.dpid': 4,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 5,
  'dst.dpid': 4,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 5,
  'dst.dpid': 1,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 5,
  'dst.dpid': 6,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0.006184605131622841,
  'free_bandwidth': None},
 {'src.dpid': 2,
  'dst.dpid': 6,
  'delay': None,
  'packet_loss': None,
  'lin

In [12]:
link_with_port_mn_to_hmap()

{('s1', 'h1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 'h1',
  'port1': 1,
  'port2': 0},
 ('h1', 's1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 'h1',
  'node2': 's1',
  'port1': 0,
  'port2': 1},
 ('s1', 'h1000'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 'h1000',
  'port1': 2,
  'port2': 0},
 ('h1000', 's1'): {'max_queue_size': 1000,
  'use_htb': True,
  'node1': 'h1000',
  'node2': 's1',
  'port1': 0,
  'port2': 2},
 ('s1', 's2'): {'bw': 106,
  'delay': '63ms',
  'loss': 3,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 's2',
  'port1': 3,
  'port2': 3},
 ('s2', 's1'): {'bw': 106,
  'delay': '63ms',
  'loss': 3,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's2',
  'node2': 's1',
  'port1': 3,
  'port2': 3},
 ('s1', 's3'): {'bw': 60,
  'delay': '28ms',
  'loss': 4,
  'max_queue_size': 1000,
  'use_htb': True,
  'node1': 's1',
  'node2': 's3',
  'port1': 4,
  'port2': 3},


In [13]:
def get_inter_group_edges_link_traffic():
    '''
        Only get bandwidth and link usage of
    '''
    inter_group_edges = get_inter_group_edges(graph)
    deltal_port_stat = get_all_delta_port_stat()
    link_with_port = link_with_port_mn_to_hmap()
    
    inter_port_stat = []
    for ige in inter_group_edges:
        node1 = ige[0]
        port1 = link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port1']
        node2 = ige[1]
        port2 = link_with_port[(f's{ige[0]}', f's{ige[1]}')]['port2']

        node1_port_stat = deltal_port_stat.get((node1, port1))
        node2_port_stat = deltal_port_stat.get((node2, port2))
        
        # ref:
        '''
            bandwidth = min(src_free_bandwidth, dst_free_bandwidth)
            link_usage = min(src_link_usage, dst_link_usage)
            
            link usage = delta upload+download min(src_dpid_port, dst_dpid_port)
            min()
        '''
        node1_traffic = node1_port_stat['tx_bytes'] + node1_port_stat['rx_bytes']
        node2_traffic = node2_port_stat['tx_bytes'] + node2_port_stat['rx_bytes']
        link_traffic = (min(node1_traffic, node2_traffic)) / (8*1000000)
        inter_port_stat.append({
            'src.dpid': node1,
            'dst.dpid': node2,
            'link_usage': link_traffic,
        })
    return inter_port_stat
get_inter_group_edges_link_traffic()

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]


[{'src.dpid': 1, 'dst.dpid': 2, 'link_usage': 0.00029875},
 {'src.dpid': 1, 'dst.dpid': 5, 'link_usage': 0.000333875},
 {'src.dpid': 2, 'dst.dpid': 1, 'link_usage': 0.00029875},
 {'src.dpid': 5, 'dst.dpid': 1, 'link_usage': 0.000333875},
 {'src.dpid': 5, 'dst.dpid': 4, 'link_usage': 0.000316875},
 {'src.dpid': 4, 'dst.dpid': 5, 'link_usage': 0.000316875}]

In [26]:
link_traffic_multi_controller = get_link_traffic_multi_controller()
link_traffic_inter_domain = get_inter_group_edges_link_traffic()

link_traffic_all_net = link_traffic_inter_domain + link_traffic_multi_controller
link_traffic_all_net

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]


[{'src.dpid': 1, 'dst.dpid': 2, 'link_usage': 0.00028525},
 {'src.dpid': 1, 'dst.dpid': 5, 'link_usage': 0.00025325},
 {'src.dpid': 2, 'dst.dpid': 1, 'link_usage': 0.00028525},
 {'src.dpid': 5, 'dst.dpid': 1, 'link_usage': 0.00025325},
 {'src.dpid': 5, 'dst.dpid': 4, 'link_usage': 0.0002655},
 {'src.dpid': 4, 'dst.dpid': 5, 'link_usage': 0.0002655},
 {'src.dpid': 6,
  'dst.dpid': 2,
  'delay': 26.875853538513184,
  'packet_loss': 0.1,
  'link_usage': 0.0011192538307794554,
  'free_bandwidth': None},
 {'src.dpid': 6,
  'dst.dpid': 5,
  'delay': 62.66427040100098,
  'packet_loss': 0.375,
  'link_usage': 0.0007994670219853557,
  'free_bandwidth': None},
 {'src.dpid': 3,
  'dst.dpid': 1,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 3,
  'dst.dpid': 4,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_bandwidth': None},
 {'src.dpid': 5,
  'dst.dpid': 4,
  'delay': None,
  'packet_loss': None,
  'link_usage': 0,
  'free_

In [31]:
link_with_port_mn_to_hmap()['s1', 's2']['bw']

106

In [37]:
# def get_multi_domain_link_info():
# Only get link_usage
link_traffic_multi_controller = get_link_traffic_multi_controller()
link_traffic_inter_domain = get_inter_group_edges_link_traffic()
link_traffic_all_net = link_traffic_inter_domain + link_traffic_multi_controller
    
# get packetloss+delay directly from mininet
link_ping_stat = rq.get('http://0.0.0.0:8000/link_ping_stat').json()

ltan_hmap = {}
lps_hmap = {}

for d in link_traffic_all_net:
    key = (d['src.dpid'], d['dst.dpid'])
    ltan_hmap[key] = d

for d in link_ping_stat:
    key = (d['src.host'], d['dst.host'])
    lps_hmap[key] = d

lwp_hmap = link_with_port_mn_to_hmap()

link_quality = []
for key, lps_value in lps_hmap.items():
    ltan_value = ltan_hmap.get(key)
    lwp_value = lwp_hmap.get((f's{key[0]}',  f's{key[1]}'))
    print(lwp_value)
    # if lps_value == None: continue  
    link_quality.append({
        'src.dpid': key[0],
        'dst.dpid': key[1],
        'packet_loss': lps_value.get('packet_loss', None),
        'delay': lps_value.get('delay', None),
        'bandwidth': lwp_value.get('bw', 1),
        'link_usage': ltan_value.get('link_usage', 0),
        'link_utilization': ltan_value.get('link_usage', 0) / lwp_value.get('bandwidth', 1) * 100,
    })

        
# get_multi_domain_link_info()

[(1, 2), (1, 5), (2, 1), (5, 1), (5, 4), (4, 5)]
{'bw': 149, 'delay': '26ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's2', 'node2': 's6', 'port1': 4, 'port2': 3}
{'bw': 149, 'delay': '26ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's6', 'node2': 's2', 'port1': 3, 'port2': 4}
{'bw': 60, 'delay': '28ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's1', 'node2': 's3', 'port1': 4, 'port2': 3}
{'bw': 60, 'delay': '28ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's3', 'node2': 's1', 'port1': 3, 'port2': 4}
{'bw': 105, 'delay': '44ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's5', 'node2': 's4', 'port1': 4, 'port2': 4}
{'bw': 105, 'delay': '44ms', 'loss': 4, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's4', 'node2': 's5', 'port1': 4, 'port2': 4}
{'bw': 90, 'delay': '43ms', 'loss': 3, 'max_queue_size': 1000, 'use_htb': True, 'node1': 's1', 'node2': 's5', 'port1': 5, 'port2': 3}
{'bw': 90

In [38]:
link_quality

[{'src.dpid': 2,
  'dst.dpid': 6,
  'packet_loss': 0.07317073170731708,
  'delay': 52.063,
  'bandwidth': 149,
  'link_usage': 0.001119626791051923,
  'link_utilization': 0.11196267910519231},
 {'src.dpid': 6,
  'dst.dpid': 2,
  'packet_loss': 0.07317073170731708,
  'delay': 52.063,
  'bandwidth': 149,
  'link_usage': 0.001119626791051923,
  'link_utilization': 0.11196267910519231},
 {'src.dpid': 1,
  'dst.dpid': 3,
  'packet_loss': 0.07804878048780488,
  'delay': 56.055,
  'bandwidth': 60,
  'link_usage': 0,
  'link_utilization': 0.0},
 {'src.dpid': 3,
  'dst.dpid': 1,
  'packet_loss': 0.07804878048780488,
  'delay': 56.055,
  'bandwidth': 60,
  'link_usage': 0,
  'link_utilization': 0.0},
 {'src.dpid': 5,
  'dst.dpid': 4,
  'packet_loss': 0.08536585365853659,
  'delay': 88.049,
  'bandwidth': 105,
  'link_usage': 0,
  'link_utilization': 0.0},
 {'src.dpid': 4,
  'dst.dpid': 5,
  'packet_loss': 0.08536585365853659,
  'delay': 88.049,
  'bandwidth': 105,
  'link_usage': 0,
  'link_util